## El primer gran proyecto: ¡Profesionalmente Tú!

### Y, uso de la herramienta.

### Pero primero: presentamos Pushover

Pushover es una herramienta práctica para enviar notificaciones push a tu teléfono.

¡Es facilísima de configurar e instalar!

Simplemente visita https://pushover.net/, crea una cuenta gratuita y genera tus claves API.

Como señaló el estudiante Ron (¡gracias, Ron!), hay dos tokens que se pueden crear en Pushover:
1. El token de usuario, que se obtiene en la página principal de Pushover.
2. El token de aplicación, que se obtiene al ir a https://pushover.net/apps/build y crear una aplicación.

(Esto te permite organizar tus notificaciones push en diferentes aplicaciones en el futuro).

Agrega a tu archivo `.env`:
```
PUSHOVER_USER=pon_tu_token_de_usuario_aquí
PUSHOVER_TOKEN=pon_tu_token_de_aplicación_aquí
```

E instala la aplicación Pushover en tu teléfono.

In [8]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [9]:
# El inicio usual

load_dotenv(override=True)
openai = OpenAI()

In [10]:
# Para pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [11]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [13]:
push("HOLA!!")

Push: HOLA!!


In [14]:
def record_user_details(email, name="Nombre no proporcionado", notes="not provided"):
    push(f"Registrando interés de {name} con email {email} y notas {notes}")
    return {"recorded": "ok"}

In [15]:
def record_unknown_question(question):
    push(f"Registrando pregunta no respondida: {question}")
    return {"recorded": "ok"}

In [16]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Utilice esta herramienta para registrar que un usuario está interesado en estar en contacto y proporcionó una dirección de correo electrónico.",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "La dirección de correo electrónico de este usuario"
            },
            "name": {
                "type": "string",
                "description": "El nombre del usuario, si lo proporcionó"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Cualquier información adicional sobre la conversación que merezca ser registrada para dar contexto"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [17]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Siempre use esta herramienta para registrar cualquier pregunta que no se pueda responder, ya que no sabía la respuesta",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "La pregunta que no se pudo responder"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [18]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [19]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Utilice esta herramienta para registrar que un usuario está interesado en estar en contacto y proporcionó una dirección de correo electrónico.',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'La dirección de correo electrónico de este usuario'},
     'name': {'type': 'string',
      'description': 'El nombre del usuario, si lo proporcionó'},
     'notes': {'type': 'string',
      'description': 'Cualquier información adicional sobre la conversación que merezca ser registrada para dar contexto'}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': 'Siempre use esta herramienta para registrar cualquier pregunta que no se pueda responder, ya que no sabía la respuesta',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 

In [16]:
# Esta función puede tomar una lista de llamadas a herramientas y ejecutarlas. ¡Este es el IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Herramienta llamada: {tool_name}", flush=True)

        # ¡EL GRAN IF !!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [20]:
globals()["record_unknown_question"]("esta es una pregunta realmente difícil")

Push: Registrando pregunta no respondida: esta es una pregunta realmente difícil


{'recorded': 'ok'}

In [21]:
# Esta es una forma más elegante de evitar el IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Herramienta llamada: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [22]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Juan Gabriel Gomila"

In [28]:
system_prompt = f"""Estás actuando como {name}. Estás respondiendo preguntas en el sitio web de {name}, en particular preguntas relacionadas con la carrera, los antecedentes, las habilidades y la experiencia de {name}.
Tu responsabilidad es representar a {name} en las interacciones en el sitio web con la mayor fidelidad posible.
Se te proporciona un resumen de los antecedentes y el perfil de LinkedIn de {name} que puedes usar para responder preguntas.
Sé profesional y atractivo, como si hablaras con un cliente potencial o un futuro empleador que haya visitado el sitio web.
Si no sabes la respuesta a alguna pregunta, usa la herramienta record_unknown_question para registrar la pregunta que no pudiste responder, incluso si se trata de algo trivial o no relacionado con tu carrera.
Si el usuario participa en una conversación, intenta que se ponga en contacto por correo electrónico; pídele su correo electrónico y regístralo con la herramienta record_user_details."""

system_prompt += f"\n\n## Resumen:\n{summary}\n\n## LinkedIn Perfil:\n{linkedin}\n\n"
system_prompt += f"En este contexto, chatea con el usuario, siempre con el personaje {name}."

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # Esta es la llamada a la LLM - nota que pasamos el json de las herramientas

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason

        # Si la LLM quiere llamar a una herramienta, la llamamos!

        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [30]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


Herramienta llamada: record_unknown_question
Push: Registrando pregunta no respondida: ¿Qué coche conduces?
Herramienta llamada: record_user_details
Push: Registrando interés de Nombre no proporcionado con email juangabriel@frogames.es y notas not provided
Herramienta llamada: record_user_details
Push: Registrando interés de Juan con email juangabriel@frogames.es y notas Usuario interesado en contactar


## Y ahora, la implementación

Este código está en `app.py`

La implementación se realizará en HuggingFace Spaces. Gracias, estudiante Robert M, por mejorar estas instrucciones.

Antes de empezar: recuerda actualizar los archivos del directorio "me" (tu perfil de LinkedIn y summary.txt) para que se muestren tus credenciales.

Comprueba también que no haya ningún archivo README en el directorio 1_foundations. Si lo hay, elimínalo. El proceso de implementación crea un nuevo archivo README en este directorio.

1. Visita https://huggingface.co y crea una cuenta.
2. En el menú Avatar, en la esquina superior derecha, selecciona "Tokens de acceso". Selecciona "Crear nuevo token". Asígnale permisos de escritura.
3. Tome este token y agréguelo a su archivo .env: `HF_TOKEN=hf_xxx`. Si no se detecta durante la implementación, consulte la nota a continuación.
4. Desde la carpeta 1_foundations, introduzca `uv run gradio deploy`. Si por alguna razón aún le solicita que introduzca su token HF, interrúmpalo con Ctrl+C y ejecute `uv run dotenv -f ../.env run -- uv run gradio deploy`, lo que obliga a que todas sus claves se configuren como variables de entorno.
5. Siga sus instrucciones: asígnele el nombre "career_conversation", especifique app.py, elija cpu-basic como hardware, confirme que es necesario proporcionar secretos, proporcione su clave de API de OpenAI, su usuario y token de Pushover, y confirme que no se permiten las acciones de GitHub. Nota adicional sobre el token HuggingFace

Un par de estudiantes han mencionado que HuggingFace no detecta su token, a pesar de estar en el archivo .env. Prueba lo siguiente:
1. Reinicia el cursor.
2. Vuelve a ejecutar load_dotenv(override=True) y usa una nueva terminal (el botón + en la esquina superior derecha de la terminal).
3. En la terminal, ejecuta esto antes de la implementación de gradio: `$env:HF_TOKEN = "hf_XXXX"`.
Gracias, James y Martins, por estos consejos.

#### Más información sobre estos secretos:

Si no entiendes qué sucede con estos secretos, solo te pide que introduzcas el nombre y el valor de la clave para cada uno de ellos. Por ejemplo, escribirías:
`OPENAI_API_KEY`
Seguido de:
`sk-proj-...`

Si no quieres configurar los secretos de esta manera o si algo sale mal, no hay problema: puedes cambiarlos más tarde:

1. Inicia sesión en el sitio web de HuggingFace.
2. Ve a tu perfil a través del menú "Avatar" en la esquina superior derecha.
3. Selecciona el espacio que has implementado.
4. Haz clic en la rueda de Ajustes en la esquina superior derecha.
5. Puedes desplazarte hacia abajo para cambiar tus secretos, eliminar el espacio, etc.

#### ¡Y ya deberías estar implementado!

Aquí está el mío: https://huggingface.co/spaces/ed-donner/Career_Conversation

Acabo de recibir una notificación push de que un estudiante me preguntó cómo puede convertirse en presidente de su país. 😂😂

Para más información sobre la implementación:

https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces

Para eliminar tu espacio en el futuro:

1. Inicia sesión en HuggingFace.
2. En el menú Avatar, selecciona tu perfil.
3. Haz clic en el espacio y selecciona la rueda de configuración en la esquina superior derecha.
4. Desplázate hasta la sección Eliminar en la parte inferior.
5. ADEMÁS: borra el archivo README que Gradio haya creado dentro de la carpeta 1_foundations (de lo contrario, no te hará las preguntas la próxima vez que implementes Gradio).

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Ejercicio</h2>
            <span style="color:#ff7800;">• Ante todo, ¡impleméntalo tú mismo! Es una herramienta real y valiosa: el futuro currículum.<br/>
• A continuación, mejora los recursos: añade un mejor contexto sobre ti. Si conoces RAG, añade una base de conocimientos sobre ti.<br/>
• ¡Añade más herramientas! Podrías tener una base de datos SQL con preguntas y respuestas comunes que el LLM pueda leer y escribir.<br/>
• Incorpora al Evaluador del último laboratorio y añade otros patrones de Agentic.
</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Implicaciones Comerciales</h2>
            <span style="color:#00bfff;">Aparte de lo obvio (tu alter ego profesional), esto tiene aplicaciones comerciales en cualquier situación en la que necesites un asistente de IA con experiencia en el dominio y capacidad para interactuar con el mundo real.
            </span>
        </td>
    </tr>
</table>